In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
301,This ranks way up there on my top list of wors...,negative
653,"One of the funniest, most romantic, and most m...",positive
755,"The tighter the drama, the better the film of ...",positive
796,"""Don't Drink the Water"" is an unbelievably bad...",negative
218,"I've gotta say, I usually like horror movies t...",negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

Error during text normalization: 
**********************************************************************
  Resource stopwords not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('stopwords')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load corpora/stopwords

  Searched in:
    - 'C:\\Users\\sreed/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\sreed\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************



LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\sreed/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\sreed\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [5]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [ ]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [13]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
301,this ranks way up there on my top list of wors...,0
653,"one of the funniest, most romantic, and most m...",1
755,"the tighter the drama, the better the film of ...",1
796,"""don't drink the water"" is an unbelievably bad...",0
218,"i've gotta say, i usually like horror movies t...",0


In [ ]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
print(y_train)

267    1
687    0
258    1
596    1
605    1
      ..
871    0
1      1
704    1
529    1
786    0
Name: sentiment, Length: 375, dtype: int64


In [8]:
pip install dagshub

  Using cached gql-3.5.3-py2.py3-none-any.whl (74 kB)
  Using cached treelib-1.8.0-py3-none-any.whl (30 kB)
  Using cached pathvalidate-3.3.1-py3-none-any.whl (24 kB)
  Using cached dacite-1.6.0-py3-none-any.whl (12 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\sreed\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [18]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/SREEDHA96/mlops_capstone_project.mlflow')
dagshub.init(repo_owner='SREEDHA96', repo_name='mlops_capstone_project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-08-09 22:17:06,083 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/SREEDHA96/mlops_capstone_project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "SREEDHA96/mlops_capstone_project"

2025-08-09 22:17:06,089 - INFO - Initialized MLflow to track repo "SREEDHA96/mlops_capstone_project"


Repository SREEDHA96/mlops_capstone_project initialized!

2025-08-09 22:17:06,092 - INFO - Repository SREEDHA96/mlops_capstone_project initialized!


<Experiment: artifact_location='mlflow-artifacts:/0e4a9df31b834c85b5be33ef9199a619', creation_time=1754757087301, experiment_id='0', last_update_time=1754757087301, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [21]:
import mlflow
import logging
import os
import time
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        # mlflow.sklearn.log_model(model, "model")
        joblib.dump(model,"model.pkl")
        mlflow.log_artifact("model.pkl")
        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-08-09 22:22:23,863 - INFO - Starting MLflow run...
2025-08-09 22:22:24,424 - INFO - Logging preprocessing parameters...
2025-08-09 22:22:25,676 - INFO - Initializing Logistic Regression model...
2025-08-09 22:22:25,678 - INFO - Fitting the model...
2025-08-09 22:22:25,728 - INFO - Model training complete.
2025-08-09 22:22:25,729 - INFO - Logging model parameters...
2025-08-09 22:22:26,078 - INFO - Making predictions...
2025-08-09 22:22:26,083 - INFO - Calculating evaluation metrics...
2025-08-09 22:22:26,126 - INFO - Logging evaluation metrics...
2025-08-09 22:22:31,103 - INFO - Saving and logging the model...
2025-08-09 22:22:33,256 - INFO - Model training and logging completed in 8.83 seconds.
2025-08-09 22:22:33,263 - INFO - Accuracy: 0.696
2025-08-09 22:22:33,270 - INFO - Precision: 0.72
2025-08-09 22:22:33,274 - INFO - Recall: 0.6
2025-08-09 22:22:33,278 - INFO - F1 Score: 0.6545454545454545


🏃 View run enchanting-mule-623 at: https://dagshub.com/SREEDHA96/mlops_capstone_project.mlflow/#/experiments/0/runs/2a546b1142564147b059d42ecf9b0b36
🧪 View experiment at: https://dagshub.com/SREEDHA96/mlops_capstone_project.mlflow/#/experiments/0
